In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user
!python3 --version

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/local/lib/python3.9 1

In [ ]:
%pip install SimpleITK==2.4.0
%pip install numpy==1.26.4
%pip install pyradiomics==3.1.0
%pip install six==1.17.0

In [ ]:
%%writefile code.py

import radiomics
import numpy as np
import SimpleITK as sitk
import radiomics.featureextractor
import os
import six

imagePath = '/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_flair.nii.gz'
image = sitk.ReadImage(imagePath)
maskPath ='/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_tumor.nii.gz'
mask = sitk.ReadImage(maskPath)

settings = {}
settings['kernelRadius'] = 5
settings['voxelBatch'] = 1024
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(**settings)

featureMap = extractor.execute(image, mask, voxelBased=True)

extractor.disableAllFeatures()
extractor.enableFeaturesByName(firstorder=['Energy'])

for featureName, featureValue in six.iteritems(featureMap):
  if isinstance(featureValue, sitk.Image):
    sitk.WriteImage(featureValue, '/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/nrrd/%s/%s.nrrd' % ('BraTS2021_00000', featureName))
    print('Computed %s, stored as "/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/nrrd/%s/%s.nrrd"' % (featureName, 'BraTS2021_00000', featureName))
  else:
    print('%s: %s' % (featureName, featureValue))

In [ ]:
!python3 code.py